## Libraries

In [23]:
!pip3 install numpy
!pip3 install pandas
!pip3 install numpy
!pip3 install sklearn
!pip3 install statsmodels
!pip install matplotlib

import matplotlib.pyplot as plt

import sklearn
import time
import random
import copy
import json

import pandas as pd # data manipulation library
import numpy as np # math library

import sklearn.metrics as sklm # metrics
import statsmodels as sm # statistical models
from sklearn.model_selection import GridSearchCV

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Utils

In [24]:
PATH = "./"

In [25]:
class WalkingForwardTimeSeriesSplit():
  def __init__(self, n_splits):
    self.n_splits = n_splits
  
  def get_n_splits(self, X, y, groups):
    return self.n_splits
  
  def split(self, X, y=None, groups=None):
    n_samples = len(X)
    k_fold_size = n_samples // self.n_splits
    indices = np.arange(n_samples)

    margin = 0
    for i in range(self.n_splits):
      start = i * k_fold_size
      stop = start + k_fold_size
      mid = int(0.8 * (stop - start)) + start
      yield indices[start: mid], indices[mid + margin: stop]

In [26]:
def clean_cv_results(cv_results):
  res = copy.deepcopy(cv_results)
  for key in res:
    if type(res[key]) != list: 
      res[key] = res[key].tolist()

  return res

In [27]:
def retrieve_data(flow_interval):
    path = "{0}dataset/dataset_flow_{1}.csv".format(PATH, flow_interval)
    print(PATH)
    data = pd.read_csv(path, ';')
    
    data['Flow'].apply(int)
    data['AveSpeed'].apply(float)
    data['Density'].apply(float)
    data['Sunday'].apply(int)
    data['Monday'].apply(int)
    data['Tuesday'].apply(int)
    data['Wednesday'].apply(int)
    data['Thursday'].apply(int)
    data['Friday'].apply(int)
    data['Saturday'].apply(int)
      
    return data

In [28]:
def print_json (obj):
  print(json.dumps(obj, sort_keys=True, indent=4))

In [29]:
def store(obj, path, name):
  with open("{0}{1}/{2}.json".format(PATH, path, name), 'w') as json_file:
    json.dump(obj, json_file, sort_keys=True, indent=4)

In [30]:
def store_results ():
  name = int(time.time())
  
  result_data['meta'] = {
    "SEEABLE_PAST": SEEABLE_PAST,
    "PREDICT_IN_FUTURE": PREDICT_IN_FUTURE,
    "FLOW_INTERVAL": FLOW_INTERVAL,
    "N_SPLITS": N_SPLITS,
  }

  store(result_data, "results", name)

  slim_result_data = copy.deepcopy(result_data)
  for model in slim_result_data['results']:
      del slim_result_data['results'][model]['raw']

  store(slim_result_data, "results", "{0}_slim".format(name))

In [31]:
def store_comparisons (title):
  name = str(int(time.time()))
  
  j = copy.deepcopy(comparison_data)

  store(j, "results/comparison", "{0}_{1}".format(title, name))
    
  for i in range(len(j)):
    for model in j[i]['results']:
      del j[i]['results'][model]['raw']

  store(j, "results/comparison", "{0}_{1}_slim".format(title, name))

In [32]:
def generate_dataset(data, useB, n_steps, n_future):
  """ Generate Dataset
  
  Generate a dataset provided a sequence. Reshape the sequence in rolling intervals from [samples, timesteps] into 
  [samples, timesteps, features] and split the sequence. The split the sequence in rolling intervals with a corresponding value 
  like the example bellow.

  Ex: split_sequence([1, 2, 3, 4, 5], 3) #([[1, 2, 3], [2, 3, 4]], [4, 5])
  
  Arguments:
    raw_seq: the sequence to reshape.
    useB: if the dataset is more complex or not.
    n_steps: size of the rolling interval
    n_future: the distance to the interval the value should be.  
  """

  sequence = np.array(data if useB else data['Flow'])

  n = len(sequence)
  X, Y = list(), list()

  for i in range(n):
    j = i + n_steps
    k = j + n_future

    if k >= n:
      break

    seq_x, seq_y = sequence[i:j], sequence[k]
    X.append(seq_x)	
    Y.append(seq_y[0] if useB else seq_y)

  X, Y = np.array(X), np.array(Y)	
  
  if not useB:
    X = X.reshape((X.shape[0], X.shape[1], 1))

  return X, Y

In [33]:
def evaluate (expected, observed, times, name):
  """ Evaluate Sessions
  
  Evaluate models by RMSE, NRMSE, MAE, HR, PRE. It will store the 
  results in a object and return it.
  
  Arguments:
    expected: an array of expected instances of each 
      train&test session.
    observed: an array of observed instances of each 
      train&test session.
    times: an array of the time of each train&test session.
    name: the name of the model
  """
  n = len(expected)
  flatten = lambda l : [i for sl in l for i in sl]
  
  # Make the arrays serializable
  expected = list(map(list, expected))
  observed = list(map(list, observed))
  
  for i in range(n):
    expected[i] = list(map(float, expected[i]))
    observed[i] = list(map(float, observed[i]))
  
  raw = evaluate_raw(expected, observed, times)
  
  #n_buckets = len(raw['PRE'])
  #_pre = [[pre[i] for pre in raw['PRE']] for i in range(n_buckets)]
  
  eva = {
    'TIME': int(sum(times)),
    'RMSE': float(np.mean(raw['RMSE'])),
    'MAE': float(np.mean(raw['MAE'])),
    'Kappa': float(np.mean(raw['Kappa'])),
    'HR': float(np.mean(raw['HR'])),
    'has_negative': (min(flatten(observed)) < 0),
    'raw': raw
  }
  
  print(f"\n{name} Final Result:")
  print(f"\tTotal Time: {eva['TIME']}s")
  print(f"\tRMSE: {eva['RMSE']}")
  print(f"\tMAE: {eva['MAE']}")
  print(f"\tKappa: {eva['Kappa']}")
  print(f"\tHit Ratio: {eva['HR'] * 100}%")
    
  return eva

In [34]:
def evaluate_raw (expected, observed, times):
  """ Evaluate Raw Sessions 
  
  Evaluate each of the train&test sessions by RMSE, NRMSE, MAE, HR, PRE. 
  It will store the results in a object and return it.
  
  Arguments:
    expected: an array of expected instances of each train&test session.
    observed: an array of observed instances of each train&test session.
    times: an array of the time of each train&test session.
  """
  
  n = len(expected)

  for i in range(n):
    observed[i] = [0 if np.isnan(o) else o for o in observed[i]]

  for i in range(n):
    observed[i] = [max(o, 0) for o in observed[i]]
  
  raw = {
    'expected': expected,
    'observed': observed,
    'TIME': times,
    'RMSE': [0] * n,
    'MAE': [0] * n,
    'Kappa': [0] * n,
    'HR': [0] * n,
  }
  
  for i in range(n):
    Y = expected[i]
    Y_hat = observed[i]
    time = times[i]

    raw['RMSE'][i] = np.sqrt(sklm.mean_squared_error(Y, Y_hat))
    raw['MAE'][i] = sklm.mean_absolute_error(Y, Y_hat)
    raw['Kappa'][i] = skllm.kappa(Y, Y_hat)
    raw['HR'][i] = evaluate_precision_hit_ratio(Y, Y_hat)
    
    if VERBOSITY:
      print(f"({i+1}/{n}) Test Size: {len(Y)}, Time: {time}s")
      print(f"\tRMSE: {raw['RMSE'][i]}")
      print(f"\tMAE: {raw['MAE'][i]}")
      print(f"\tKappa: {raw['Kappa'][i]}")
      print(f"\tHit Ratio: {raw['HR'][i] * 100}%")

  return raw

In [35]:
def evaluate_precision_hit_ratio (Y, Y_hat):
  """ Trend Prediction Ratio Calculation
  
  Calculates the ratio of up/down prediction.
  
  Arguments:
    Y: the expected dataset.
    Y_hat: the observed dataset.
  """
  
  cnt = 0
  
  for i in range(len(Y)):
    if i < N_FUTURE:
      continue
      
    exp = Y[i] - Y[i - N_FUTURE]
    obs = Y_hat[i] - Y[i - N_FUTURE]
    
    if exp * obs > 0:
      cnt += 1
    
  return cnt / len(Y)

## Testing

In [36]:
# Model Parameters
SEEABLE_PAST = 480 # in minutes
PREDICT_IN_FUTURE = 60 # in minutes
FLOW_INTERVAL = 150 # the interval size for each flow
N_SPLITS = 8

In [37]:
# Derivated Model Parameters
N_STEPS = SEEABLE_PAST * 60 // FLOW_INTERVAL # the number of flows to see in the past
N_FUTURE = PREDICT_IN_FUTURE * 60 // FLOW_INTERVAL # how much in the future we want to predict (0 = predict the flow on the next FLOW_INTERVAL minutes)
DAY_SIZE = (24 * 60 * 60) // FLOW_INTERVAL  
WEEK_SIZE = (7 * 24 * 60 * 60) // FLOW_INTERVAL
VERBOSITY = True

In [38]:
def naive (X, Y):
  global result_data

  X = X.reshape(X.shape[0], X.shape[1])

  name = "Naive"
  cv = WalkingForwardTimeSeriesSplit(n_splits=N_SPLITS)
  expected, observed, times = [], [], []

  for train_index, test_index in cv.split(X):
    X_test = X[test_index]
    Y_test = Y[test_index]
  
    start_time = time.time()
    Y_hat = [x[-1] for x in X_test]
    end_time = time.time()
    
    expected.append(Y_test)
    observed.append(Y_hat)
    times.append(end_time - start_time)
    
  result_data['results'][name] = evaluate(expected, observed, times, name)
  store(result_data['results'][name], "results/grid", "{0}_{1}".format(name, PREDICT_IN_FUTURE))

In [39]:
global result_data
  
result_data = {
    'results': {},
    'meta': {
      'SEEABLE_PAST': SEEABLE_PAST,
      'PREDICT_IN_FUTURE': PREDICT_IN_FUTURE,
      'FLOW_INTERVAL': FLOW_INTERVAL,
      'N_SPLITS': N_SPLITS,
    }
}

data = retrieve_data(FLOW_INTERVAL)
X_a, Y_a = generate_dataset(data, False, N_STEPS, N_FUTURE)
naive(X_a, Y_a)

./


FileNotFoundError: [Errno 2] No such file or directory: './dataset/dataset_flow_150.csv'

In [287]:
def plot_prediction (Y, Y_hat, title):
  """ Plot Prediction
  
  Plot the prediction (Flow x Time) of what was expected and what
  was predicted.
  """

  for i in range(len(Y)):
    name = f"{title} ({str(i+1).zfill(2)} of {len(Y)})"
    path = f"plots/prediction/{name}"
    
    plt.plot(Y[i])
    plt.plot(Y_hat[i])
    plt.title(f"Predição do Modelo {title}")
    plt.ylabel('Fluxo')
    plt.xlabel('Tempo')
    plt.legend(['esperado', 'observado'], loc='upper left')
    plt.rcdefaults()

    plt.savefig(path + ".png", bbox_inches='tight')
    # plt.savefig(path + ".pdf")

    plt.close('all')

In [288]:
expected = result_data['results']['Naive']['raw']['expected']
observed = result_data['results']['Naive']['raw']['observed']

plot_prediction(expected, observed, "Naive")